In [ ]:
 #✅ Binary Loan Default Risk Prediction Using Feature Engineering, SMOTE, and Ensemble Learning on ARTHIMPACT Dataset

# Step 1: Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

# Step 2: Load and merge data
df_loan = pd.read_csv("loan_data.csv")
df_person = pd.read_csv("personal_data.csv")

# Merge datasets on person_id
df = pd.merge(df_loan, df_person, on='person_id', how='inner')

# Step 3: Data cleaning and feature creation
# Convert dates
df['dob'] = pd.to_datetime(df['dob'], errors='coerce')
df['start_date'] = pd.to_datetime(df['start_date'], errors='coerce')

# Age calculation
df['age'] = (pd.to_datetime('today') - df['dob']).dt.days // 365

# Convert dpd to numeric
df['dpd'] = pd.to_numeric(df['dpd'], errors='coerce')
df.dropna(subset=['dpd'], inplace=True)

# Step 4: Label target using dpd > 15 as high risk
threshold = 15
df['is_high_risk'] = (df['dpd'] > threshold).astype(int)

# Step 5: Feature Engineering
# Create new ratio features
df['loan_to_age_ratio'] = df['amount'] / (df['age'] + 1)
df['tenure_to_amount_ratio'] = df['tenure'] / (df['amount'] + 1)
df['is_large_loan'] = (df['amount'] > df['amount'].median()).astype(int)

# Step 6: Drop irrelevant columns
df.drop(columns=['loan_id', 'name', 'address', 'pan', 'dob', 'start_date', 'dpd'], inplace=True)

# Step 7: One-hot encoding for categorical variables
df = pd.get_dummies(df, columns=['gender', 'status'], drop_first=True)

# Step 8: Handle missing values
df.fillna(df.median(numeric_only=True), inplace=True)

# ✅ Check class distribution to detect imbalance
print("Class Distribution (is_high_risk):\n", df['is_high_risk'].value_counts())
sns.countplot(x='is_high_risk', data=df)
plt.title("Class Distribution")
plt.show()

# Step 9: Train-test split
X = df.drop(columns=['is_high_risk', 'person_id'])
y = df['is_high_risk']

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

# ✅ Step 10: Handle class imbalance using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

# Step 11: Define models
models = {
    'XGBoost': XGBClassifier(learning_rate=0.1, n_estimators=100, max_depth=4,
                              subsample=0.8, colsample_bytree=0.8,
                              objective='binary:logistic', eval_metric='logloss', use_label_encoder=False),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42)
}

# Step 12: Train and evaluate each model
for name, model in models.items():
    print(f"\n=== {name} ===")
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")

    # ROC Curve
    fpr, tpr, _ = roc_curve(y_test, y_proba)
    plt.figure(figsize=(6, 5))
    plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc_score(y_test, y_proba):.2f})")
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title(f"ROC Curve - {name}")
    plt.legend(loc="lower right")
    plt.grid()
    plt.tight_layout()
    plt.show()

# Step 13: Feature importance from XGBoost
xgb_model = models['XGBoost']
feat_importance = pd.Series(xgb_model.feature_importances_, index=X.columns)
plt.figure(figsize=(10,6))
feat_importance.sort_values().plot(kind='barh')
plt.title('Feature Importance (XGBoost)')
plt.tight_layout()
plt.show()